# SC1235 Introduction to Medical Image Analysis Using Convolutional Neural Networks

<div id="toc"></div>

## 4. Preparing for Image Segmentation

In [ ]:
from keras.layers import InputLayer, Conv2D, MaxPool2D, Flatten, Dense, UpSampling2D, LocallyConnected2D
from keras.models import Model, Sequential
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

### Check if a GPU is indeed available

Sometimes, it is useful to be able to check that tensorflow (which is used internally by keras here) is able to see a real GPU. If tensorflow falls back to the CPU, your code will still work, but be *much* slower.

In [ ]:
from tensorflow.python.client import device_lib
# print all devices visible to tensorflow 
print(device_lib.list_local_devices())

The first thing one needs to prepare is how to access the data:
* separation of training & test data
* random access to parts of the data (e.g. single images out of a large database)
* random permutations (shuffling)

In this notebook, we will just load data that we have prepared.  In practice, one might be interested in
* [pydicom](https://pydicom.github.io) for loading DICOM image data
* [HDF5 and h5py](https://www.h5py.org) for efficient storage of large binary data
* [numpy.random.permutation](https://docs.scipy.org/doc/numpy/reference/generated/numpy.random.permutation.html) comes in very handy for shuffling large arrays

> If you are already capable of python programming, here are some more hints how you could improve on the approach of this notebook:
> * Create a generator function that loads one batch of size `batch_size` and returns it.
> * Track which images you have already drawn, so that you can start over after one epoch (one epoch is one run through all images) with a fresh permutation.
> * Call the `fit_generator()` function instead of `fit()`.  Read the Keras documentation for this at [keras.io](http://keras.io).
>
> If you still have time and motivation, implement the same using a HDF5 file storage rather than plain disk storage.

### Loading the Data

We have prepared a set of abdominal CT slices, strongly downsampled, completely anonymized, together with segmentation labels which we will inspect shortly.  The following cell just downloads the data to the machine this notebook runs on.

In [ ]:
!test -e tmp_slices.npz || curl -L "https://drive.google.com/uc?export=download&id=1R2-H0dhhrj6XNK7Q-MazIWGeFDOf6Zya" --output tmp_slices.npz

Split into separate training and test sets.

* Training converges with about 200 slices.
* The initial results when training with 700 slices are terrible. (Exercise: Why?)

In [ ]:
TRAINING_SLICE_COUNT = 300

In [ ]:
loaded = np.load('tmp_slices.npz')

x_train = loaded['x_train'][:TRAINING_SLICE_COUNT]
y_train = loaded['y_train'][:TRAINING_SLICE_COUNT]

x_test = loaded['x_train'][TRAINING_SLICE_COUNT:]
y_test = loaded['y_train'][TRAINING_SLICE_COUNT:]

assert len(x_train) == len(y_train)

### Inspecting the Demo Data
The PNGs used for this hands-on contain abdominal CT scans showing the liver, resampled to 4mm voxel size and after applying a liver HU window (centered at 20 HU, width 450 HU).  The corresponding masks contain values 0 (background), 1 (liver), 2..3 (different classes of lesions):

In [ ]:
np.unique(y_test)

In [ ]:
example_test_slice = 1800

f, ax = plt.subplots(1, 2, figsize = (9, 5))
imgplot = ax[0].imshow(x_test[example_test_slice])
ax[0].set_title('orig')
imgplot = ax[1].imshow(y_test[example_test_slice])
ax[1].set_title('mask')
plt.show()

For now, we will start with a binary segmentation problem (0: background, 1: liver), so we will remove the lesion labels / turn them into liver (value 1).

In [ ]:
# remove the lesion labels (values 2..3)
y_train_binary = y_train.clip(0, 1)
y_test_binary = y_test.clip(0, 1)

f, ax = plt.subplots(1, 2, figsize = (9, 5))
imgplot = ax[0].imshow(x_test[example_test_slice])
ax[0].set_title('orig')
imgplot = ax[1].imshow(y_test_binary[example_test_slice])
ax[1].set_title('mask')
plt.show()

# 5. Segmentation: Auto Encoder (AE)-Style
* We define short functions that return a model.
* The first is a simple architecture that collapses and expands an image into the desired mask, similar to an Auto Encoder (AE).
* The second is the famous U-Net.

Further reading: Variational AE

In [ ]:
def getModel(_filters=32, filters_add=0, _kernel_size=(3,3), _padding='same', _activation='relu', _kernel_regularizer=None, _final_layer_nonlinearity='sigmoid'):
    model = Sequential()
    # We are indifferent about the xy size, but accept only one channel (gray value images). This has the consequence that debugging sizes gets harder.
    model.add(InputLayer(input_shape=(None,None,1))) 
    
    model.add(Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, name='firstConvolutionalLayer'))
    model.add(Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(MaxPool2D())

    model.add(Conv2D(filters=_filters+2*filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(Conv2D(filters=_filters+2*filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(UpSampling2D())

    model.add(Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(Conv2D(filters=_filters+filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(UpSampling2D())

    model.add(Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))
    model.add(Conv2D(filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer))

    model.add(Conv2D(1, kernel_size=(1,1), activation=_final_layer_nonlinearity))
    return model

## Convolution Mode 'valid' vs. 'same' / Automatic Padding

In case you use convolutions with `'valid'` padding, the input size needs to be padded before submitting so that still an output corresponding to the full slice (or the patch size, respectively) is generated. The number of pixels to pad depends on the network architecture. There is an excellent technical report detailing the arithmetics to calculate the receptive field and required padding from a network definition at https://arxiv.org/pdf/1603.07285.pdf. Another description is at https://medium.com/@Synced/a-guide-to-receptive-field-arithmetic-for-convolutional-neural-networks-42f33d4378e0.

We will explore the difference later in the context of _overlapping tiles segmentation_.
The function is somewhat hard-coded for the network architecture above: it pads the input with a fixed amount of voxels in case the model gives an output that is differently sized from the inmput.

In [ ]:
def pad_image_for_model(model, input_image):
    '''Determine the necessary amount of padding
    (difference between input and output size of the model)
    and apply it to an ndarry with one or more images.'''
    
    padding = 0
    if model.get_layer('firstConvolutionalLayer').padding == 'valid':
        padding = 20 # WARNING: Hard-coded for above architecture!

        # determine in which dimension to apply this padding
        ndim_padding = []
        if np.ndim(input_image) > 2:
            # do not pad along batch dimension (if present)
            ndim_padding.append((0, 0))
        ndim_padding.append((padding, padding)) # pad above/below image (y dimension)
        ndim_padding.append((padding, padding)) # pad left/right of image (x dimension)
        if np.ndim(input_image) > 3:
            # do not pad along channel dimension (if present)
            ndim_padding.append((0, 0))
        
        input_image = np.lib.pad(input_image, ndim_padding,
                                 #'constant', constant_values = 0)
                                 'reflect')

    return input_image, padding

For our small data, it is possible to train a classifier that takes full slices. However, when dealing with input of arbitrary size, this will no longer work. Images have to be partitioned, and the individual results need to be stitched into the final output. This only works when padding the input with blank or other voxels to achieve the desired output size.

### Experiment 1: 'valid' Convolutions

Get a model with padding of 'valid', i.e. the size will shrink and a proper amount of voxels need to be added prior to processing.

In [ ]:
modelValid = getModel(_padding = 'valid')

modelValid.compile(loss='binary_crossentropy', optimizer='adam')
print("Model parameters: {0:,}".format(modelValid.count_params()))

### Experiment 2: 'same' Convolutions

Get a model with `same` padding, i.e. the input is padded with zeros before each convolution such that an output size equal to the input is achieved. 

In [ ]:
modelSame = getModel()

modelSame.compile(loss='binary_crossentropy', optimizer='adam')
print("Model parameters: {0:,}".format(modelSame.count_params()))

## Executing code during training

This is a small interludium section that isn't directly related to the experiments around the overlapping tile concept.

In Keras (and also other frameworks), code can be executed with every iteration/batch/... Keras makes this particularly easy by offering "callbacks" where you can override one or more functions to execute your code. We will use this to generate images of the learning success after each epoch.

In [ ]:
from keras.callbacks import Callback

class VisualHistory(Callback):
    def on_train_begin(self, logs={}):
        # also show initial prediction
        plot_prediction(self.model, example_test_slice)
    
    def on_epoch_end(self, batch, logs={}):
        # show prediction after every training epoch
        plot_prediction(self.model, example_test_slice)
        
vh_callback = VisualHistory()

Once a model is trained, you'll want to evaluate it. Predicting using a given model and then plotting one slice is implemented in this function. 

In [ ]:
def do_prediction(model, input_image, verbose = False):
    # first do padding of full slice
    input_image, padding = pad_image_for_model(model, input_image)
    
    # add batch and channel dimensions (network expects 4D arrays)
    input_array = input_image[np.newaxis,:,:,np.newaxis]
    if verbose:
        print("input shape:", input_array.shape)

    y_predicted = model.predict(input_array)
    if verbose:
        print("output shape:", y_predicted.shape)

    return input_image, input_array, y_predicted, padding

def plot_prediction(model, pred_slice_index):
    # get single slice
    input_image    = x_test[pred_slice_index]
    # could use y_train_binary here for the first half of the notebook, but in the end we want to see the lesion
    reference_mask = y_test[pred_slice_index]

    input_image, input_array, y_predicted, padding = do_prediction(model, input_image)
    
    padded_extent = np.array([0, input_array.shape[2], input_array.shape[1], 0]) - 0.5 - padding

    # display prediction for inspection
    f, ax = plt.subplots(1, 5 if padding else 4, figsize = (11 if padding else 8, 3), sharey = True)
    ax[0].imshow(x_test[pred_slice_index])
    ax[0].set_title('orig')
    if padding:
        ax[1].imshow(input_array[0,:,:,0], extent = padded_extent)
        ax[1].set_title('padded input')
    ax[-2].imshow(y_predicted[0,:,:,0])
    ax[-2].set_title('predicted mask')
    ax[-3].imshow(reference_mask.clip(0,1))
    ax[-3].set_title('reference mask')
    ax[-1].imshow(reference_mask.clip(0,1) - y_predicted[0,:,:,0])
    ax[-1].set_title('(ref - predicted)')
    ax[0].set_ylim(*padded_extent[2:])
    plt.show()

## Training
* As the arrays we created before are 3-dimensional (no channel for grey images), we have to add one dimension to make it compatible with the ConvNet.
* About 100 epochs lead to a pretty well-performing net. On an average CPU, one iteration takes about 10-15 sec. On GPU, this is much faster (increase the batch size also, to avoid unneccessary GPU memory transfers)
* With Batch Normalisation and PReLU, the number of parameters gets much larger, and training takes much longer. 
    * Does the result warrant the wait?
    * Explain!
* Callbacks enable better logging. 
    * We can add the TensorBoard logging mechanism. 
    * TensorBoard needs to be started externally, pointing to the log directory, which defaults to `./logs`.

In [ ]:
# This is for _padding = 'same'
historySame = modelSame.fit(x_train[...,np.newaxis],
                            y_train_binary[...,np.newaxis],
                            batch_size=20, epochs=5, callbacks=[vh_callback])

In [ ]:
# This is for _padding = 'valid' and 'reflect' padding
historyValid = modelValid.fit(np.lib.pad(x_train[...,np.newaxis],
                                         [(0,0), (20,20), (20,20), (0,0)], 'reflect'),
                              y_train_binary[...,np.newaxis],
                              batch_size=20, epochs=5, callbacks=[vh_callback])

## Prediction
Let's look at the prediction from some more example slices, but let's only use the `x_test` slices that we did not use for training. (In a real scenario, we would to the separation of training & test data on the level of patients, *before* extracting slices, and we'd also have a validation set.)

In [ ]:
slice_indices = np.random.choice(x_test.shape[0], 6)

### Evaluate model with 'valid' convolutions

In [ ]:
for a in slice_indices:
    plot_prediction(modelValid, a)

### Evaluate model with 'same' convolutions

In [ ]:
for a in slice_indices:
    plot_prediction(modelSame, a)

### Tile-based Prediction

Let's predict the image divided into upper and lower half, and then in full. Note that the network could also predict smaller or larger tile sizes.  Such an approach is necessary for large images, or with volumetric data and 3D convolutions, when it is not possible to have the whole image in (GPU) memory at the same time.

In [ ]:
# hand-picked slice useful to visualize benefits of tiled approach
example_test_slice = 505 - TRAINING_SLICE_COUNT
assert example_test_slice > 0, 'when increasing TRAINING_SLICE_COUNT, please pick a new example slice as well'


def plot_prediction_tiled(model, pred_slice_index, tile = None):
    # get single slice
    input_image    = x_train[pred_slice_index]
    reference_mask = y_train_binary[pred_slice_index]

    # first do padding of full slice
    input_image, padding = pad_image_for_model(model, input_image)
    
    # add batch and channel dimensions (network expects 4D arrays)
    input_array = input_image[np.newaxis,:,:,np.newaxis]
    print("padded input shape:", input_array.shape)

    # then cut the specified tile box (plus padding)
    if tile is not None: # (default is full slice)
        tile = np.asarray(tile)
        assert tile.shape == (2, 2)
        padded_tile = tile.copy()
        padded_tile[:,1] += 2*padding
        input_array = input_array[:,
                                  padded_tile[0][0]:padded_tile[0][1],
                                  padded_tile[1][0]:padded_tile[1][1],
                                  :]
        reference_mask = reference_mask[tile[0][0]:tile[0][1],
                                        tile[1][0]:tile[1][1]]
        print("tiled padded shape:", input_array.shape)

    y_predicted = model.predict(input_array)
    print("output shape:", y_predicted.shape)

    padded_extent = np.array([0,input_array.shape[2],input_array.shape[1],0]) - 0.5 - padding
    orig_extent = np.array([0,reference_mask.shape[1],reference_mask.shape[0],0]) - 0.5
    if tile is not None:
        padded_extent[:2] += tile[1,0]
        padded_extent[2:] += tile[0,0]
        orig_extent[:2] += tile[1,0]
        orig_extent[2:] += tile[0,0]

    # display prediction for inspection
    f, ax = plt.subplots(1, 4 if padding else 3, figsize = (14 if padding else 12, 3), sharey = True)
    ax[0].imshow(x_train[pred_slice_index])
    ax[0].set_title('orig')
    if padding:
        ax[1].imshow(input_array[0,:,:,0], extent = padded_extent)
        ax[1].set_title('padded input')
    ax[-2].imshow(y_predicted[0,:,:,0], extent = orig_extent)
    ax[-2].set_title('predicted mask')
    ax[-1].imshow(reference_mask, extent = orig_extent)
    ax[-1].set_title('reference mask')
    ax[0].set_ylim(*padded_extent[2:])
    plt.show()

In [ ]:
tile = [[0,42],[0,76]]
plot_prediction_tiled(modelSame, example_test_slice, tile)
tile = [[42,76],[0,76]]
plot_prediction_tiled(modelSame, example_test_slice, tile)
plot_prediction(modelSame, example_test_slice)

In [ ]:
tile = [[0,42],[0,76]]
plot_prediction_tiled(modelValid, example_test_slice, tile)
tile = [[42,76],[0,76]]
plot_prediction_tiled(modelValid, example_test_slice, tile)
plot_prediction(modelValid, example_test_slice)

## Regularisation
* Regularisation should improve convergence. Let's try and add some Batch Normalisation first. Batch Normalisation intends to normalise the input to a (convolutional) layer, so that the values in the resulting feature maps don't get exessively large. 
    * Keras offers BatchNorm layers.
    * Include one before each convolutional layer.
* Another regularisation measure is to choose better activation functions. Probabilistic Rectified Linear Units (PReLU) crop negative values to a small epsilon, but route through any value greater than zero.
    * In Keras, activation functions are selected through the `activation=[softmax|elu|selu|relu|tanh|sigmoid|hard_sigmoid|linear]` parameter to a layer, each in quotes.
    * PReLU is one of the advanced activation functions that need to be added as a layer. It has many "trainable" parameters. How many? Why?
    * How many parameters does ReLU have?
* Lastly, L1 and L2 norm can be used as additional constraints on weights, biases, and activations.
    * In Keras, this normalisation is again a parameter to the layer initialisation, using `kernel_regularizer=[l1(0.01)|l2(0.01)|l1_l2(0.01)]`.
    * You have to `from keras.regularizers import l1, l2, l1_l2` to enable this functionality.
    * You can also explore `bias_regularizer` and `activity_regularizer`.
* Make your life easier by extracting a block: Conv -- Relu -- Batch Normalization. Then play with the options -- but carefully: When you just switch everything "on", chances are you will get bad results...

In [ ]:
# We will use the following block to generate the regularisation block

from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU, PReLU

def addConvBN(model, filters=32, kernel_size=(3,3), batch_norm=True, activation='prelu', padding='same', kernel_regularizer=None, name = None):
    if batch_norm:
        model.add(BatchNormalization())
    if activation == 'prelu':
        model.add(Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation='linear', kernel_regularizer=kernel_regularizer, name = name))
        model.add(PReLU())
    elif activation == 'lrelu':
        model.add(Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation='linear', kernel_regularizer=kernel_regularizer, name = name))
        model.add(LeakyReLU())
    else:
        model.add(Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation=activation, kernel_regularizer=kernel_regularizer, name = name))

In [ ]:
# batch norm model
def getBNModel(_filters=32, _filters_add=0, _kernel_size=(3,3), _padding='same', _activation='prelu', _kernel_regularizer=None, _final_layer_nonlinearity='sigmoid', _num_classes=1):
    model = Sequential()
    
    # this is really ugly, but TensorFlow's batch normalization
    # currently has a limitation that it cannot work on unknown input sizes,
    # so we need to get the height & width of our training data:
    h, w = x_train.shape[1:]
    if _padding == 'valid':
        model.add(InputLayer(input_shape = (h+40, w+40, 1)))
    elif _padding == 'same':
        model.add(InputLayer(input_shape = (h, w, 1)))

    addConvBN(model, filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, name='firstConvolutionalLayer')
    addConvBN(model, filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    model.add(MaxPool2D())

    addConvBN(model, filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    addConvBN(model, filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    model.add(MaxPool2D())

    addConvBN(model, filters=_filters+2*_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    addConvBN(model, filters=_filters+2*_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    model.add(UpSampling2D())

    addConvBN(model, filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    addConvBN(model, filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    model.add(UpSampling2D())

    addConvBN(model, filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)
    addConvBN(model, filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer)

    model.add(Conv2D(_num_classes, kernel_size=(1,1), activation=_final_layer_nonlinearity))
    return model

### Batch Norm with PReLU

In [ ]:
# This network uses the PReLU layer.
# Note the number of parameters when executing model.summary().
modelValidBN = getBNModel(_padding='valid')
modelValidBN.compile(loss='binary_crossentropy', optimizer='adam')
modelValidBN.summary()

In [ ]:
padded_x_train, padding = pad_image_for_model(modelValidBN, x_train[...,np.newaxis])
historyValidBN = modelValidBN.fit(padded_x_train,
                                  y_train_binary[...,np.newaxis],
                                  batch_size=10, epochs=5, callbacks=[vh_callback])

In [ ]:
fig,ax = plt.subplots(figsize=(18, 4), dpi= 80, facecolor='w', edgecolor='k')
ax.plot(historyValid.history['loss'], label = 'without BN')
ax.plot(historyValidBN.history['loss'], label = 'with batch normalization')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.grid()
ax.legend(loc = 'best')
plt.show()

Alternatively, we could use padded convolutions again:

In [ ]:
modelSameBN = getBNModel(_padding='same')
modelSameBN.compile(loss='binary_crossentropy', optimizer='adam')
history = modelSameBN.fit(x_train[...,np.newaxis], y_train_binary[...,np.newaxis],
                          batch_size=10, epochs=5, callbacks=[vh_callback]) 

### Batch Norm with ReLU

In [ ]:
# Now compare with a model with ReLU (instead of PReLu)
model = getBNModel(_padding='valid', _activation='relu')
model.compile(loss='binary_crossentropy', optimizer='adam')
print("Model parameters: {0:,}".format(model.count_params()))

In [ ]:
padded_x_train, padding = pad_image_for_model(model, x_train[...,np.newaxis])
history = model.fit(padded_x_train, y_train_binary[...,np.newaxis],
                    batch_size=10, epochs=75, callbacks=[vh_callback]) 

### Regularisation using L1/L2 norm on weights

In [ ]:
from keras.regularizers import l1, l2, l1_l2

model = getModel(_padding='valid', _kernel_regularizer = l2(0.001))
model.compile(loss='binary_crossentropy', optimizer='adam')
print("Model parameters: {0:,}".format(model.count_params()))

In [ ]:
padded_x_train, padding = pad_image_for_model(model, x_train[...,np.newaxis])
history = model.fit(padded_x_train,
                    y_train_binary[...,np.newaxis],
                    batch_size=10, epochs=75, callbacks=[vh_callback]) 

# 6. Loss functions
* Loss functions take the predicted output, `y_pred`, and the expected output, `y_train`, and calculate their distance. The result is the minimization target.

## Loss functions: Final layer non-linearity dependency
* We have worked with binary crossentropy. See [this blog series](http://neuralnetworksanddeeplearning.com/chap3.html) for a comment:
    > When should we use the cross-entropy instead of the quadratic cost?<p>In fact, the cross-entropy is nearly always the better choice, provided the output neurons are sigmoid neurons.
* Experiment with different loss functions: `mean_squared_error | logcosh | binary_crossentropy | cosine_proximity`
* Experiment with different final layer nonlinearities: `softmax | elu | selu | relu | tanh | sigmoid | hard_sigmoid | linear`

## Loss Functions: Jaccard
Generally considered a powerful loss is also Jaccard loss; it provides larger errors and therefore more stable gradients close to the solution. $l_j = \frac{\sum |A*B|}{\sum |A| +\sum |B| -\sum |A*B|}$

In practice, we also have to prevent division by zero. The following code uses a smoothing term to avoid exploding or disapearing gradients.

In [ ]:
from keras import backend as K

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    The jaccard distance loss is useful for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

### Train model with Jaccard loss.

In [ ]:
model = getModel()
model.compile(loss=jaccard_distance_loss, optimizer='adam')
print("Model parameters: {0:,}".format(model.count_params()))

padded_x_train, padding = pad_image_for_model(model, x_train[...,np.newaxis])
history = model.fit(padded_x_train, 
                    y_train_binary[...,np.newaxis], 
                    batch_size=10, epochs=75, callbacks=[vh_callback]) 

## Loss Functions: Dice
* For segmentation, the Dice loss is also very common. $l_d = 2*\sum \frac{|A*B|} {\sum A^2 + \sum B^2}$

NB: Jaccard and Dice are very similar overlap measures and can easily be computed from each other (bijection):
<img src="images/jaccard_vs_dice.png">

In [ ]:
import keras.backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    @url: https://gist.github.com/wassname/7793e2058c5c9dacb5212c0ac0b18a8a
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

### Train model with Dice loss

In [ ]:
model = getBNModel()
model.compile(loss=dice_coef_loss, optimizer='adadelta')
print("Model parameters: {0:,}".format(model.count_params()))

padded_x_train, padding = pad_image_for_model(model, x_train[...,np.newaxis])
history = model.fit(padded_x_train,
                    y_train_binary[...,np.newaxis],
                    batch_size=10, epochs=75, callbacks=[vh_callback]) 

# Segmentation using a U-Net
* U-Nets are characterized by a downsampling path and an upsamling path, which allow for a pixel-wise output.
* Skip connections are used between them in order to make it easier for the network to retain fine details.
* Below is a diagram of the U-Net we will create now. You'll learn how to create it, too.
<img src="images/U-net_4_levels.png" alt="U-Net diagram"/>

In [ ]:
# We will use this to generate the regularisation block for the sequential model.
def addConvBNSequential(model, filters=32, kernel_size=(3,3), batch_norm=True, activation='prelu', padding='same', kernel_regularizer=None, name=None):
    if batch_norm:
        model = BatchNormalization()(model)
    if activation == 'prelu':
        model = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation='linear', kernel_regularizer=kernel_regularizer, name=name)(model)
        model = PReLU()(model)
    elif activation == 'lrelu':
        model = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation='linear', kernel_regularizer=kernel_regularizer, name=name)(model)
        model = LeakyReLU()(model)
    else:
        model = Conv2D(filters=filters, kernel_size=kernel_size, padding=padding, activation=activation, kernel_regularizer=kernel_regularizer, name=name)(model)
    return model

In [ ]:
# Creates a small U-Net.
from keras.layers import Input, concatenate
def get_batchnorm_unet(_filters=32, _filters_add=0, _kernel_size=(3,3), _padding='same', _activation='prelu', _kernel_regularizer=None, _final_layer_nonlinearity='sigmoid', _batch_norm=True):

    h, w = x_train.shape[1:]
    if _padding == 'valid':
        input_layer = Input(shape = (h+40, w+40, 1))
    elif _padding == 'same':
        input_layer = Input(shape = (h, w, 1))

    x0 = addConvBNSequential(input_layer, filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm, name = 'firstConvolutionalLayer')
    x0 = addConvBNSequential(x0,          filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x1 = MaxPool2D()(x0)
    
    x1 = addConvBNSequential(x1,          filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x1 = addConvBNSequential(x1,          filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x2 = MaxPool2D()(x1)
    
    x2 = addConvBNSequential(x2,          filters=_filters+2*_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x2 = addConvBNSequential(x2,          filters=_filters+2*_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x3 = UpSampling2D()(x2)
    
    x3 = concatenate([x1,x3])
    x3 = addConvBNSequential(x3,          filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x3 = addConvBNSequential(x3,          filters=_filters+_filters_add, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x4 = UpSampling2D()(x3)
    
    x4 = concatenate([x0,x4])
    x4 = addConvBNSequential(x4,          filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)
    x4 = addConvBNSequential(x4,          filters=_filters, kernel_size=_kernel_size, padding=_padding, activation=_activation, kernel_regularizer=_kernel_regularizer, batch_norm=_batch_norm)

    output_layer = Conv2D(1, kernel_size=(1,1), activation=_final_layer_nonlinearity)(x4)
    
    model = Model(input_layer, output_layer)
    return model

In [ ]:
model = get_batchnorm_unet(_activation='relu', _batch_norm=True)
model.compile(loss='binary_crossentropy', optimizer='adam')
print("Model parameters: {0:,}".format(model.count_params()))

In [ ]:
# It gets increasingly interesting to plot the architecture.
# (1) plotting to PNG image file
from keras.utils import plot_model
plot_model(model, to_file='U-Net.png', show_shapes=False, show_layer_names=True)

from IPython.display import Image
Image(filename = 'U-Net.png')

In [ ]:
# (2) plotting to SVG vector graphics format
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

SVG(model_to_dot(model).create(prog='dot', format='svg'))

In [ ]:
history = model.fit(x_train[...,np.newaxis],
                    y_train_binary[...,np.newaxis],
                    batch_size=10, epochs=75, callbacks=[vh_callback]) 

In [ ]:
model = get_batchnorm_unet(_activation='relu', _batch_norm=True)
model.compile(loss=dice_coef_loss, optimizer='adam')
print("Model parameters: {0:,}".format(model.count_params()))
history = model.fit(x_train[...,np.newaxis],
                    y_train[...,np.newaxis],
                    batch_size=10, epochs=75, callbacks=[vh_callback]) 

## Multi-Label Segmentation

In [ ]:
# Convert the labels into a one-hot representation
from keras.utils.np_utils import to_categorical

# Convert to uint8 data and find out how many labels there are
t = y_train.astype(np.uint8)
t_max = int(np.max(y_test))
print("Range of values: [0, {}]".format(t_max))
y_train_one_hot = to_categorical(t, num_classes=t_max+1).reshape((y_train.shape)+(t_max+1,))
print("Shape before: {}; Shape after: {}".format(y_train.shape, y_train_one_hot.shape))

# The liver neuron should also be active for lesions within the liver
liver = np.max(y_train_one_hot[:,:,:,1:], axis=3)
y_train_one_hot[:,:,:,1] = liver

In [ ]:
model = getBNModel(_num_classes=4)
model.compile(loss=dice_coef_loss, optimizer='adadelta')
print("Model parameters: {0:,}".format(model.count_params()))
padded_x_train, padding = pad_image_for_model(model, x_train[...,np.newaxis])
history = model.fit(padded_x_train,
                    y_train_one_hot,
                    batch_size=10, epochs=5, callbacks=[vh_callback]) 

### Assignment: Extend the plot function to handle multiple classes.
Then, activate the visualization callback in the training again. Try to find a slice with more than one output class to see the success.

In [ ]:
%%javascript
// This code generates the table of contents at the top of the notebook
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')